---
title: "Data Cleaning"
format:
    html: 
        code-fold: false
---

## Introduction and Motivation

With this stage, I will merge and clean the data I collected from the Consumer Expenditure Survey via the U.S. Bureau of Labor Statistics. The data is organized in a cryptic way. For example, data on income, expenses, and individuals' demographics are separated into multiple csv files each. Within the csv files, there are numerous encoded variables that need to be deciphered via a data dictionary before I can begin selecting the potentially relevant columns. I will do that for each of the relevant csv files so that there will be a clean data file which I can begin to do exploratory data analysis, and eventually unsupervised and supervised learning on. 

## Overview of Methods
For this portion, I will be utilizing standard data cleaning tasks such as missing value handling, outlier value handling, duplicates removal, merging datasets, one-hot encoding, and filtering for relevant columns. Missing value handling allows for the missing values either to be imputed so the data record can be included, or if there are too many missing values in the records, the record will be dropped altogether. Outlier value handling is important because it helps identify unusual data points that can lead to significant problems in exploratory data analysis, unsupervised learning and supervised learning. Duplicates removal reduce the redundancy in the dataset while column filtering purposely narrows down the dataset to reduce noise. One-hot encoding is the process of converting categorical variables into binary format. For example, the variable 'sex' in the data will have values male and female. Through this process, two new columns will be created, corresponding each of the classes. In each of the resulting columns about male and female, the value will either be 1 (meaning yes) or 0 (meaning no). One-hot encoding is crucial for machine learning algorithms as it improves prediction by allowing ML models to understand and use the data more easily. All of these methods will aid the accuracy of the supervised machine learning models I will train.

The expense and income data files are presented on a per instance basis. For example, each expense and income payment for each individual are recorded as an item. In order to make it tidy, I will be aggregating based on the individual identifier 'NEWID' to get total income and expenses in the given year.  Additionally, there will be merging of data frames to form data frames for income, expenses and characteristics. 

## Code 

In [57]:
# Load in necessary packages
import pandas as pd
import numpy as np

For this project, I will be handling the data from the **Consumer Expenditure Survey**. For each of the datasets used from this survey, I had to go through the corresponding data dictionary (stored in an Excel) to select potentially relevant features.
We begin with the income data.

In [58]:
#  Import data for income
income_1_df = pd.read_csv("../../data/raw-data/itii232.csv")
income_2_df = pd.read_csv("../../data/raw-data/itii233.csv")
income_3_df = pd.read_csv("../../data/raw-data/itii234.csv")
income_4_df = pd.read_csv("../../data/raw-data/itii241.csv")

In [59]:
# Examine one of the dataframes
print(income_1_df.describe)

<bound method NDFrame.describe of           NEWID  REFMO  REFYR     UCC  PUBFLAG VALUE_  IMPNUM        VALUE
0       5090604      1   2023  900030        2    NaN       1  3169.833300
1       5090604      1   2023  900030        2    NaN       2  3169.833300
2       5090604      1   2023  900030        2    NaN       3  3169.833300
3       5090604      1   2023  900030        2    NaN       4  3169.833300
4       5090604      1   2023  900030        2    NaN       5  3169.833300
...         ...    ...    ...     ...      ...    ...     ...          ...
330445  5366911      5   2023  980071        2    NaN       1   820.250000
330446  5366911      5   2023  980071        2    NaN       2   250.000000
330447  5366911      5   2023  980071        2    NaN       3   100.000000
330448  5366911      5   2023  980071        2    NaN       4   294.666667
330449  5366911      5   2023  980071        2    NaN       5   160.250000

[330450 rows x 8 columns]>


Now I filter for the relevant columns in the income dataframes. From the data collection stage, we already know that each of dataframes has 8 columns. We want the following columns:
- The variable "NEWID" represent the unique identifier for the survey participant. 
- The feature 'UCC' stands for Universal Classification Code which correpond to goods and services and other things that can be bought or sold. The values under variable "UCC" correspond to items that would increases or decreases to the individuals' net worth. 
- The variable "VALUE" indicate the absolute value of the change in net worth. The other 5 variables only represent data reelvant to the survey process so we subset the dataframes for those 3 columns.

In [60]:
income_columns_keep = ['NEWID', 'UCC', 'VALUE']

income_1_df = income_1_df[income_columns_keep]
print(income_1_df.shape)

income_2_df = income_2_df[income_columns_keep]
print(income_2_df.shape)

income_3_df = income_3_df[income_columns_keep]
print(income_3_df.shape)

income_4_df = income_4_df[income_columns_keep]
print(income_4_df.shape)

(330450, 3)
(330840, 3)
(322320, 3)
(325200, 3)


Next, we want to find the unqiue "UCC" values to see if we have to deal with decreases in net worth. 

In [61]:

# Initialize list that stores all unique values of 'UCC' column
all_UCC_unique = []

# Function that prints the unique values in a particular column and returns the list
def find_unique_UCC_values(df, column_name, unique_UCC_values):

  unique_values = df[column_name].unique()
  for value in unique_values:
    if value not in all_UCC_unique:
        unique_UCC_values.append(value)
        
  
find_unique_UCC_values(income_1_df, 'UCC', all_UCC_unique)
find_unique_UCC_values(income_2_df, 'UCC', all_UCC_unique)
find_unique_UCC_values(income_3_df, 'UCC', all_UCC_unique)
find_unique_UCC_values(income_4_df, 'UCC', all_UCC_unique)

print(all_UCC_unique)

[900030, 900170, 900180, 980000, 980071, 800940, 900000, 900160, 900150, 900090, 900190, 900200, 900210, 900120, 900140]


By referring to the data dictionary, I found that the "UCC" values are mostly associated with increases, except for 800940 which represents deductions for social security. 
There is some overlap between them. For example, 980071 represent income after taxes. Here I want to only focus on pre-tax income for simplicity's sake. Therefore we filter for the following:
- 900030: Social Security and railroad retirement income
- 900170: Retirement, survivors, disability income
- 900180: Interest and dividends
- 980000: Income before taxes
- 800940: Deductions for Social Security
- 900150: Food stamps

The following codes correspond to income that is lumped into 980000: Income before taxes
- 900160: Self-employment income
- 900000: Wages and salaries 
- 900090: Supplemental security income
- 900190: Net room/rental income
- 900200: Royalty, estate, trust income
- 900210: Other regular income
- 900140: Other income

In [62]:
income_df_UCC_keep = [900030, 900170, 900180, 980000, 800940, 900150]

negation_UCC_value = 800940

# Function to filter for the 'UCC' values we want and negate if UCC = 800940
def filter_and_negate(df, negation_ucc):

  # Filter the DataFrame based on the UCC list
  filtered_df = df[df['UCC'].isin(income_df_UCC_keep)]

  # Negate the 'VALUE' column for the specific UCC
  filtered_df.loc[filtered_df['UCC'] == negation_ucc, 'VALUE'] *= -1

  return filtered_df

# Apply the function to the data frames and check the shape 
income_1_df = filter_and_negate(income_1_df, negation_UCC_value)
print(income_1_df.shape)

income_2_df = filter_and_negate(income_2_df, negation_UCC_value)
print(income_2_df.shape)

income_3_df = filter_and_negate(income_3_df, negation_UCC_value)
print(income_3_df.shape)

income_4_df = filter_and_negate(income_4_df, negation_UCC_value)
print(income_4_df.shape)


(182790, 3)
(182475, 3)
(178470, 3)
(179925, 3)


With the cleaned income dataframes, we aggregate each to find the total income per year per individual and merge all the dataframes to get the master income dataframe. Then we remove duplicates and filter outliers. I have decided to only eliminate outliers for this dataframe because I am assuming no one will be spending significantly more than they earn. Therefore, eliminating outliers for the income dataframe should be adequate.

In [63]:
# Function sums income sources based on participant ID 
def calculate_total_income(df):

# Use reset_index to make a hierarchical index a regular column
  total_income_df = df.groupby('NEWID')['VALUE'].sum().reset_index() 
  
  # Rename columns in place
  total_income_df.columns = ['id', 'total_income']
  return total_income_df


# Calculate total income for each DataFrame
total_income_df1 = calculate_total_income(income_1_df)
total_income_df2 = calculate_total_income(income_2_df)
total_income_df3 = calculate_total_income(income_3_df)
total_income_df4 = calculate_total_income(income_4_df)

# Concatenate dataframes to get total income per survey participant
total_income_df = pd.concat([total_income_df1, total_income_df2, total_income_df3, total_income_df4], axis = 0)

# Remove duplicates based on 'id' column
total_income_df.drop_duplicates(subset='id', keep='first', inplace=True)

# Remove outliers
# Calculate IQR and identify outliers
Q1 = total_income_df['total_income'].quantile(0.25)
Q3 = total_income_df['total_income'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Filter out outliers
total_income_df = total_income_df[(total_income_df['total_income'] >= lower_bound) & (total_income_df['total_income'] <= upper_bound)]

# Dataframe of income over a year
print(total_income_df.shape)

(17382, 2)


In [64]:
print(total_income_df.head)

<bound method NDFrame.head of            id  total_income
0     5090604   101692.5000
1     5090624    34467.5010
2     5090634   155839.9995
3     5090664    72695.0001
4     5090674    43196.2500
...       ...           ...
4673  5607931   222792.4995
4674  5607951    54740.0010
4675  5607961   130770.0000
4677  5608001    84775.0005
4679  5608061    65240.0010

[17382 rows x 2 columns]>


Now we handle the expenditures data. 

In [65]:
#  Import data for expenses
expense_1_df = pd.read_csv("../../data/raw-data/mtbi232.csv")
expense_2_df = pd.read_csv("../../data/raw-data/mtbi233.csv")
expense_3_df = pd.read_csv("../../data/raw-data/mtbi234.csv")
expense_4_df = pd.read_csv("../../data/raw-data/mtbi241.csv")

# Subset expense Data Frames for the relevant columns
expense_columns_keep = ['NEWID', 'SEQNO', 'UCC', 'COST']
expense_1_df = expense_1_df[expense_columns_keep]
expense_2_df = expense_2_df[expense_columns_keep]
expense_3_df = expense_3_df[expense_columns_keep]
expense_4_df = expense_4_df[expense_columns_keep]

expense_df = pd.concat([expense_1_df, expense_2_df, expense_3_df, expense_4_df], axis = 0)


By consulting the data dictionary, I located the specific files that listed the online purchases of tangible goods. 

In [66]:
# Import csv files for online purchases
specific_expense_df1 = pd.read_csv("../../data/raw-data/apb23.csv")
specific_expense_df2 = pd.read_csv("../../data/raw-data/eqb23.csv")
specific_expense_df3 = pd.read_csv("../../data/raw-data/mis23.csv")
specific_expense_df4 = pd.read_csv("../../data/raw-data/ovb23.csv")

/var/folders/sm/dt8s0j7j0glg1jfrlpndm0lw0000gn/T/ipykernel_84138/3622570946.py:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  specific_expense_df1 = pd.read_csv("../../data/raw-data/apb23.csv")


In [67]:
print(specific_expense_df1.columns)
print(specific_expense_df2.columns)
print(specific_expense_df3.columns)
print(specific_expense_df4.columns)

Index(['QYEAR', 'NEWID', 'SEQNO', 'ALCNO', 'REC_ORIG', 'MINAPPLY', 'MINA_PLY',
       'GFTCMIN', 'GFTCMIN_', 'MIN_MO', 'MIN_MO_', 'MINPURX', 'MINPURX_',
       'MINRENTX', 'MINR_NTX', 'MNAPPL1', 'MNAPPL1_', 'MNAPPL2', 'MNAPPL2_',
       'MNAPPL3', 'MNAPPL3_', 'MNAPPL4', 'MNAPPL4_', 'MNAPPL5', 'MNAPPL5_',
       'MNAPPL6', 'MNAPPL6_', 'MNAPPL7', 'MNAPPL7_', 'MNAPPL8', 'MNAPPL8_',
       'MNAPPL9', 'MNAPPL9_', 'INSTLSCR', 'INST_SCR', 'INSTLLEX', 'INST_LEX',
       'APBPURCH'],
      dtype='object')
Index(['QYEAR', 'NEWID', 'SEQNO', 'ALCNO', 'REC_ORIG', 'APPRPRYB', 'APPR_RYB',
       'SRVCMOB', 'SRVCMOB_', 'REPAIRX', 'REPAIRX_', 'APPRPB1', 'APPRPB1_',
       'APPRPB2', 'APPRPB2_', 'APPRPB3', 'APPRPB3_', 'APPRPB4', 'APPRPB4_',
       'APPRPB5', 'APPRPB5_', 'APPRPB6', 'APPRPB6_', 'APPRPB7', 'APPRPB7_',
       'APPRPB8', 'APPRPB8_', 'APPRPB9', 'EQBPURCH', 'APPRPB9_'],
      dtype='object')
Index(['QYEAR', 'NEWID', 'SEQNO', 'ALCNO', 'REC_ORIG', 'MISCCODE', 'MISC_ODE',
       'MISCMO', 'MISCMO

Filter the specific purchase Data Frames for the relevant columns so we get four dataframes of online expenses.
- 'SEQNO' is the identifier variable for the purchases and can be used to merge with expense data frames. 
- 'APBPURCH' tells us if this item was purchased online or in-person.

In [68]:
apb_columns_keep = ['NEWID', 'SEQNO', 'APBPURCH']
online_expense_df1 = specific_expense_df1[apb_columns_keep]
online_expense_df1_subset = online_expense_df1.loc[online_expense_df1['APBPURCH'] == 1]


- 'EQBPURCH' tells us if this item was bought online or in-person.

In [69]:
eqb_columns_keep = ['NEWID', 'SEQNO', 'EQBPURCH']
online_expense_df2 = specific_expense_df2[eqb_columns_keep]
online_expense_df2_subset = online_expense_df2.loc[online_expense_df2['EQBPURCH'] == 1]

- 'MISPURCH' tells us if this item was bought online or in-person.

In [70]:
mis_columns_keep = ['NEWID', 'SEQNO', 'MISPURCH']
online_expense_df3 = specific_expense_df3[mis_columns_keep]
online_expense_df3_subset = online_expense_df3.loc[online_expense_df3['MISPURCH'] == 1]

- 'OVBPURCH' tells us if this item was bought online or in-person.

In [71]:
ovb_columns_keep = ['NEWID', 'SEQNO', 'OVBPURCH']
online_expense_df4 = specific_expense_df4[ovb_columns_keep]
online_expense_df4_subset = online_expense_df4.loc[online_expense_df4['OVBPURCH'] == 1]

Now we combine the mini-dataframes to get the the online_expense_df which we will merge with the expense_df to see the dollar amount of the online purchase. Then, we aggregate based upon 'NEWID' primary key to calculate the total expense per person, total online expense and the online spending percentage. 

In [72]:
#Concatenate all online expense df
online_expense_df = pd.concat([online_expense_df1_subset, online_expense_df2_subset, online_expense_df3_subset, online_expense_df4_subset], axis = 0)

# Filter for relevant columns
online_expense_df = online_expense_df[['NEWID', 'SEQNO']]

# Add a column with imputed constant values of 1 for later merging so we know the expenses are online 
online_expense_df['Is_Online'] = 1
print("Online Expense Columns: ",online_expense_df.columns)

# Merge the two DataFrames based on 'NEWID' and 'SEQNO'
merged_expense_df = expense_df.merge(online_expense_df, on=['NEWID', 'SEQNO'], how='left')
print("Merged Expense Columns: ", merged_expense_df.columns)


# Calculate total expenses, online expenses, and online percentage for each individual
# Group by 'NEWID' column, and perform 3 aggregate functions
total_expense_df = merged_expense_df.groupby('NEWID').agg(

    id=('NEWID', 'first'),

    #create a new column, and calculates the sum of 'COST'
    total_expense=('COST', 'sum'), 

    #filter COST column to include only rows where Is_Online is 1 ( online purchases)
    online_expense=('COST', lambda x: x[merged_expense_df['Is_Online'] == 1].sum()), 

    #filter COST column for online purchases and claculates that as a % of total
    online_percentage=('COST', lambda x: x[merged_expense_df['Is_Online'] == 1].sum() / x.sum() * 100)
)

# Add a column 'bought_online' to indicate whether the person bought something online
total_expense_df['bought_online'] = (total_expense_df['online_expense'] > 0).astype(int)

# Drop duplicates
total_expense_df.drop_duplicates(subset='id', keep='first', inplace=True)

print("Total Expense Columns: ", total_expense_df.columns)
print(total_expense_df.shape)

Online Expense Columns:  Index(['NEWID', 'SEQNO', 'Is_Online'], dtype='object')
Merged Expense Columns:  Index(['NEWID', 'SEQNO', 'UCC', 'COST', 'Is_Online'], dtype='object')
Total Expense Columns:  Index(['id', 'total_expense', 'online_expense', 'online_percentage',
       'bought_online'],
      dtype='object')
(18871, 5)


Now we move on to handling characteristics data. This requires a lot of referring to the data dictionary.

In [73]:
# Import data for individual characteristics
characteristics_df_1 = pd.read_csv("../../data/raw-data/memi232.csv")
characteristics_df_2 = pd.read_csv("../../data/raw-data/memi233.csv")
characteristics_df_3 = pd.read_csv("../../data/raw-data/memi234.csv")
characteristics_df_4 = pd.read_csv("../../data/raw-data/memi241.csv")

# Filter for  relevant columns
characteristics_columns_keep = ["NEWID", "AGE", "ARM_FORC", "EARNTYPE", "EDUCA", "INCWEEKQ", 
                                "INDRETX", "JSSDEDX", "JSSDEDXM", "MARITAL", "MEMBRACE", "RC_ASIAN", "RC_BLACK", "RC_DK", "RC_NATAM",
                                "RC_OTHER", "RC_PACIL", "RC_WHITE", "SEX", "SOCRRX"]

characteristics_df_1 = characteristics_df_1[characteristics_columns_keep]
characteristics_df_2 = characteristics_df_2[characteristics_columns_keep]
characteristics_df_3 = characteristics_df_3[characteristics_columns_keep]
characteristics_df_4 = characteristics_df_4[characteristics_columns_keep]

# Combine all of them
characteristics_df = pd.concat([characteristics_df_1, characteristics_df_2, characteristics_df_3, characteristics_df_4], axis = 0)




In [74]:
# Rename columns
characteristics_df.rename(columns={"NEWID": "id", "AGE": "age", "ARM_FORC": "is_military",  "EARNTYPE": "earning_type", "EDUCA": "highest_ed_completed", 
                                   "INCWEEKQ": "num_weeks_worked_in_last_yr", "INDRETX": "deposited_money_in_retirement_this_yr", 
                                   "JSSDEDX": "income_into_SS_this_yr", "JSSDEDXM": "SS_payments_received_this_yr", "MARITAL": "marital_status", 
                                   "MEMBRACE": "race", "RC_ASIAN": "is_asian", "RC_BLACK": "is_black", "RC_DK": "race_unknown", 
                                   "RC_NATAM": "is_native_american", "RC_OTHER": "is_other_race", "RC_PACIL": "is_pacific_islander", 
                                   "RC_WHITE": "is_white", "SEX": "sex", "SOCRRX": "SS_and_railroad_retirement_income_received"}, inplace=True)


We need to do some data cleaning for the categorical variables and one-hot encode them.

In [75]:
#Data cleaning for one-hot encoding 
# Assign not military to 0
characteristics_df["is_military"] = characteristics_df['is_military'].replace(2, 0)
# Assign female to 0
characteristics_df["sex"] = characteristics_df['sex'].replace(2, 0)

# One-hot encode and add columns back to the dataframe
# Earning Type
earning_type_dummies = pd.get_dummies(characteristics_df['earning_type'], prefix='', prefix_sep='', dtype=int)
earning_type_dummies.rename(columns={
    '1.0': 'full_time_1_yr', 
    '2.0': 'part_time_1_yr', 
    '3.0': 'full_time_part_yr', 
    '4.0': 'part_time_part_yr'
}, inplace=True)
characteristics_df = pd.concat([characteristics_df, earning_type_dummies], axis=1)

# Highest Education Completed
highest_ed_dummies = pd.get_dummies(characteristics_df['highest_ed_completed'], prefix='', prefix_sep='', dtype=int)
highest_ed_dummies.rename(columns={
    '1.0': 'no_school_completed', 
    '2.0': 'grades_1-8_completed', 
    '3.0': 'high_school_no_degree', 
    '4.0': 'high_school_grad',
    '5.0': 'some_college_no_degree',
    '6.0': 'associates_degree',
    '7.0': 'bachelors_degree',
    '8.0': 'graduate_degree'
}, inplace=True)
characteristics_df = pd.concat([characteristics_df, highest_ed_dummies], axis=1)

# Marital Status
marital_dummies = pd.get_dummies(characteristics_df['marital_status'], prefix='', prefix_sep='', dtype=int)
marital_dummies.rename(columns={
    '1': 'is_married', 
    '2': 'is_widowed', 
    '3': 'is_divorced', 
    '4': 'is_separated',
    '5': 'is_single'
}, inplace=True)
characteristics_df = pd.concat([characteristics_df, marital_dummies], axis=1)

# Sex
sex_dummies = pd.get_dummies(characteristics_df['sex'], prefix='', prefix_sep='', dtype=int)
sex_dummies.rename(columns={
    '1': 'is_male', 
    '0': 'is_female'
}, inplace=True)


characteristics_df = pd.concat([characteristics_df, sex_dummies], axis=1)

In [76]:
print(characteristics_df.columns)

Index(['id', 'age', 'is_military', 'earning_type', 'highest_ed_completed',
       'num_weeks_worked_in_last_yr', 'deposited_money_in_retirement_this_yr',
       'income_into_SS_this_yr', 'SS_payments_received_this_yr',
       'marital_status', 'race', 'is_asian', 'is_black', 'race_unknown',
       'is_native_american', 'is_other_race', 'is_pacific_islander',
       'is_white', 'sex', 'SS_and_railroad_retirement_income_received',
       'full_time_1_yr', 'part_time_1_yr', 'full_time_part_yr',
       'part_time_part_yr', 'no_school_completed', 'grades_1-8_completed',
       'high_school_no_degree', 'high_school_grad', 'some_college_no_degree',
       'associates_degree', 'bachelors_degree', 'graduate_degree',
       'is_married', 'is_widowed', 'is_divorced', 'is_separated', 'is_single',
       'is_female', 'is_male'],
      dtype='object')


In [77]:
# View values per column about race
# Legend: White - 1, Black - 2, Native American - 3, Asian - 4, Pacific Islander - 5, Other - 6, Unknown - 7
race_columns = ["is_asian", "is_black", "race_unknown", "is_native_american", "is_other_race", "is_pacific_islander", "is_white"]

# One-Hot Encode Race Columns
# Replace numerical values for one-hot-encoding
characteristics_df["is_asian"] = characteristics_df['is_asian'].replace(4, 1)
characteristics_df["is_black"] = characteristics_df['is_black'].replace(2, 1)
characteristics_df["race_unknown"] = characteristics_df['race_unknown'].replace(7, 1)
characteristics_df["is_native_american"] = characteristics_df['is_native_american'].replace(3, 1)
characteristics_df["is_other_race"] = characteristics_df['is_other_race'].replace(6, 1)
characteristics_df["is_pacific_islander"] = characteristics_df['is_pacific_islander'].replace(5, 1)

# Replace NaN values with 7 (unknown race)
characteristics_df[race_columns] = characteristics_df[race_columns].fillna(7)

# Rename earning_type categories and replace numbers with categorical values
earning_dict = {1: 'full-time 1 year', 2: 'part-time 1 year', 3: 'full-time part year', 4: 'part-time part year'}
characteristics_df['earning_type'] = characteristics_df['earning_type'].replace(earning_dict)

# Rename race categories and replace numbers with categorical values
race_dict = {1: 'white', 2: 'black', 3: 'native american', 4: 'asian', 5: 'pacific islander', 6: 'other', 7: 'unknown'}
characteristics_df['race'] = characteristics_df['race'].replace(race_dict)

# Rename marital_status categories and replace numbers with categorical values
marital_dict = {1: 'married', 2: 'widowed', 3: 'divorced', 4: 'separated', 5: 'never married'}
characteristics_df['marital_status'] = characteristics_df['marital_status'].replace(marital_dict)

# Rename highest_ed_completed categories and replace numbers with categorical values
ed_dict = {1: 'no school completed', 2: 'grades 1-8', 3: 'high school no degree', 4: 'high school grad', 5: 'some college no degree', 
           6: 'associates degree', 7: 'bachelors degree', 8: 'graduate degree'}
characteristics_df['highest_ed_completed'] = characteristics_df['highest_ed_completed'].replace(ed_dict)

# Rename sex categories and replace numbers with categorical values
sex_dict = {0: 'female', 1: 'male'}
characteristics_df['sex'] = characteristics_df['sex'].replace(sex_dict)

In [78]:
print(characteristics_df.columns)

Index(['id', 'age', 'is_military', 'earning_type', 'highest_ed_completed',
       'num_weeks_worked_in_last_yr', 'deposited_money_in_retirement_this_yr',
       'income_into_SS_this_yr', 'SS_payments_received_this_yr',
       'marital_status', 'race', 'is_asian', 'is_black', 'race_unknown',
       'is_native_american', 'is_other_race', 'is_pacific_islander',
       'is_white', 'sex', 'SS_and_railroad_retirement_income_received',
       'full_time_1_yr', 'part_time_1_yr', 'full_time_part_yr',
       'part_time_part_yr', 'no_school_completed', 'grades_1-8_completed',
       'high_school_no_degree', 'high_school_grad', 'some_college_no_degree',
       'associates_degree', 'bachelors_degree', 'graduate_degree',
       'is_married', 'is_widowed', 'is_divorced', 'is_separated', 'is_single',
       'is_female', 'is_male'],
      dtype='object')


Now we merge the total income, expenses, and characteristics dataframe with one another to form the final dataframe, ces_df.

In [79]:
ces_df = total_expense_df.merge(total_income_df, on='id', how='inner')
ces_df = ces_df.merge(characteristics_df, on='id',how='left')

print(ces_df.columns)
print(ces_df.head)

Index(['id', 'total_expense', 'online_expense', 'online_percentage',
       'bought_online', 'total_income', 'age', 'is_military', 'earning_type',
       'highest_ed_completed', 'num_weeks_worked_in_last_yr',
       'deposited_money_in_retirement_this_yr', 'income_into_SS_this_yr',
       'SS_payments_received_this_yr', 'marital_status', 'race', 'is_asian',
       'is_black', 'race_unknown', 'is_native_american', 'is_other_race',
       'is_pacific_islander', 'is_white', 'sex',
       'SS_and_railroad_retirement_income_received', 'full_time_1_yr',
       'part_time_1_yr', 'full_time_part_yr', 'part_time_part_yr',
       'no_school_completed', 'grades_1-8_completed', 'high_school_no_degree',
       'high_school_grad', 'some_college_no_degree', 'associates_degree',
       'bachelors_degree', 'graduate_degree', 'is_married', 'is_widowed',
       'is_divorced', 'is_separated', 'is_single', 'is_female', 'is_male'],
      dtype='object')
<bound method NDFrame.head of             id  total_ex

Check for nulls in each column. After examining the null counts and their respective columns, there seem to be quite a few null values and several columns. I will drop them as there is a significant chunk of missing data.

In [80]:
print(ces_df.isnull().sum())
ces_df.drop(columns=['is_military', 'num_weeks_worked_in_last_yr', 'deposited_money_in_retirement_this_yr', 
                              'income_into_SS_this_yr', 'SS_payments_received_this_yr', 'SS_and_railroad_retirement_income_received'],axis=1, inplace=True)


id                                                0
total_expense                                     0
online_expense                                    0
online_percentage                                 0
bought_online                                     0
total_income                                      0
age                                               0
is_military                                   15826
earning_type                                  18886
highest_ed_completed                           6066
num_weeks_worked_in_last_yr                    6067
deposited_money_in_retirement_this_yr         35028
income_into_SS_this_yr                        18855
SS_payments_received_this_yr                  18855
marital_status                                    0
race                                              0
is_asian                                          0
is_black                                          0
race_unknown                                      0
is_native_am

In [81]:
# New null values
print(ces_df.isnull().sum())

id                            0
total_expense                 0
online_expense                0
online_percentage             0
bought_online                 0
total_income                  0
age                           0
earning_type              18886
highest_ed_completed       6066
marital_status                0
race                          0
is_asian                      0
is_black                      0
race_unknown                  0
is_native_american            0
is_other_race                 0
is_pacific_islander           0
is_white                      0
sex                           0
full_time_1_yr                0
part_time_1_yr                0
full_time_part_yr             0
part_time_part_yr             0
no_school_completed           0
grades_1-8_completed          0
high_school_no_degree         0
high_school_grad              0
some_college_no_degree        0
associates_degree             0
bachelors_degree              0
graduate_degree               0
is_marri

The columns 'total_income', 'online_percentage' and 'total_expense' both have negative values. Negative income can suggest a net loss in net worth such as expenses exceeding income. Similarly, a negative online spending percentage is spending beyond means. However, there are only 54 negative total income and 8 negative online percentage values so let's drop the rows where that occurs.

We also want to remove outliers in the dataset based on income. The rationale is that income is the main driver for expenses and therefore online spending. In other words, those who spend more online will on average have a higher income 

In [82]:
df_with_numerical_col = ces_df[['total_expense', 'online_expense', 'online_percentage', 'total_income', 'age']]
print(df_with_numerical_col.describe())

# Count negative values in each column
neg_total_income = (ces_df['total_income'] < 0).sum()
neg_online_expense = (ces_df['online_expense'] < 0).sum()

# Print the results
print("Number of negative values in 'total_income':", neg_total_income)
print("Number of negative values in 'online_expense':", neg_online_expense)

# Drop negative values
ces_df.drop(ces_df[ces_df['total_income']<0].index, inplace=True)
ces_df.drop(ces_df[ces_df['total_expense']<0].index, inplace=True)
ces_df.drop(ces_df[ces_df['online_expense']<0].index, inplace=True)

# Remove income and online expense outliers

#Set Q1 and Q3 for interquartile range
Q1_income = ces_df['total_income'].quantile(0.25)
Q3_income = ces_df['total_income'].quantile(0.75)
IQR_income = Q3_income - Q1_income

# Define outlier thresholds
lower_bound_income = Q1_income - 1.5 * IQR_income
upper_bound_income = Q3_income + 1.5 * IQR_income


# Remove outliers
ces_df_filtered = ces_df[(ces_df['total_income'] >= lower_bound_income) & (ces_df['total_income'] <= upper_bound_income)]

# Create a new column 'Income_Category' based on 'total_income' for classification task
def income_category(income):
    if income <= 50000:
        return 'Low'
    elif 50000 < income <= 100000:
        return 'Middle'
    else:
        return 'High'

ces_df_filtered['income_category'] = ces_df_filtered['total_income'].apply(income_category)


       total_expense  online_expense  online_percentage   total_income  \
count   3.991100e+04    39911.000000       39911.000000   39911.000000   
mean    1.073045e+05      124.981985           0.090174  123548.701514   
std     1.226776e+05     3402.473749           1.629428   76393.490978   
min    -6.417860e+05        0.000000         -18.103448  -47375.425500   
25%     2.083075e+04        0.000000           0.000000   64980.000000   
50%     8.383250e+04        0.000000           0.000000  109979.249400   
75%     1.463160e+05        0.000000           0.000000  169692.499500   
max     2.077078e+06   156872.000000          85.916254  349275.721500   

                age  
count  39911.000000  
mean      41.987447  
std       23.894380  
min        0.000000  
25%       22.000000  
50%       42.000000  
75%       62.000000  
max       87.000000  
Number of negative values in 'total_income': 54
Number of negative values in 'online_expense': 0


/var/folders/sm/dt8s0j7j0glg1jfrlpndm0lw0000gn/T/ipykernel_84138/1384561808.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ces_df_filtered['income_category'] = ces_df_filtered['total_income'].apply(income_category)


In [83]:
print(ces_df_filtered['income_category'].value_counts())

income_category
High      21335
Middle    11239
Low        6683
Name: count, dtype: int64


In [84]:
print(ces_df_filtered.columns)
print(ces_df_filtered.shape)

Index(['id', 'total_expense', 'online_expense', 'online_percentage',
       'bought_online', 'total_income', 'age', 'earning_type',
       'highest_ed_completed', 'marital_status', 'race', 'is_asian',
       'is_black', 'race_unknown', 'is_native_american', 'is_other_race',
       'is_pacific_islander', 'is_white', 'sex', 'full_time_1_yr',
       'part_time_1_yr', 'full_time_part_yr', 'part_time_part_yr',
       'no_school_completed', 'grades_1-8_completed', 'high_school_no_degree',
       'high_school_grad', 'some_college_no_degree', 'associates_degree',
       'bachelors_degree', 'graduate_degree', 'is_married', 'is_widowed',
       'is_divorced', 'is_separated', 'is_single', 'is_female', 'is_male',
       'income_category'],
      dtype='object')
(39257, 39)


In [85]:
# Convert to csv for access in later stages
ces_df_filtered.to_csv('../../data/processed-data/ces_clean.csv', index=False)

## Summary and Interpretation of Results:
I ended up with a dataset with the columns as presented above. We have almost 40,000 data records to work with and 40 features which should be more than enough. We have data on total income, total expenses, online shopping expendtiture, wprking information, and demographic details. Based on that, I will be addressing the following research questions:

1. Can we predict whether an individual has shopped online based on their demographic and financial information in the past year?
2. Can we predict one's online shopping expenditure based on their annual earnings and expenses?
3. Can we predict one's sex based on their demographics and shopping habits?
4. Can we predict one's employment type or education level based on demographic and financial information?

Please continue reading on the EDA page to further our exploration of this dataset.